In [1]:
import numpy as np
import random
from IPython.display import clear_output
from tqdm import tqdm
import os
from bson import ObjectId
from importlib import reload
import global_param; reload(global_param)
import functions; reload(functions)
import LCDM_stiff_Neff; reload(LCDM_stiff_Neff)
import stiff_SGWB; reload(stiff_SGWB)
from stiff_SGWB import LCDM_SG as sg
from pyDOE import lhs
import pymongo
os.environ['KMP_DUPLICATE_LIB_OK']= 'TRUE'
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["solve"]

In [2]:
'''
大范围搜索
'''
def generate_parameters_lhs():
    """
    使用拉丁超立方采样LHS生成参数样本。
    参数:
        num_samples (int): 需要生成的样本数量
    返回:
        np.ndarray: 形如 (num_samples, 5) 的数组，列顺序为 [r, n_t, kappa10, T_re, DN_re]
    """
    # 生成LHS样本，范围[0, 1)
    # lhs_samples = lhs(5, samples=num_samples, criterion='maximin')
    
    # 处理每个参数的缩放
    # r: 对数缩放 [1e-25, 1]
    log_r = np.log10(1e-25) + lhs_samples[:, 0] * (np.log10(1) - np.log10(1e-25))
    r = 10 ** log_r
    # n_t: 线性缩放 [-1, 6]
    n_t = lhs_samples[:, 1] * (6 - (-1)) + (-1)
    # kappa10: 对数缩放 [1e-7, 1e3]
    log_kappa = np.log10(1e-7) + lhs_samples[:, 2] * (np.log10(1e3) - np.log10(1e-7))
    kappa10 = 10 ** log_kappa
    # T_re: 对数缩放 [1e-3, 1e7]
    log_T = np.log10(1e-3) + lhs_samples[:, 3] * (np.log10(1e7) - np.log10(1e-3))
    T_re = 10 ** log_T
    # DN_re: 线性缩放 [0, 40]
    DN_re = lhs_samples[:, 4] * 40
    # 合并参数
    return np.column_stack((r, n_t, kappa10, T_re, DN_re))

In [19]:
lhs_samples = lhs(5, samples=100, criterion='maximin')

In [20]:
collection = db["data_test"]  
if __name__ == "__main__":
    samples = generate_parameters_lhs()
    for i in tqdm(samples):
        collection.insert_one({
            'r':i[0],
            'n_t':i[1],
            'kappa10':i[2],
            'T_re':i[3],
            'DN_re':i[4]
        })

100%|██████████| 100/100 [00:00<00:00, 3571.23it/s]


In [ ]:
import numpy as np
from bson import ObjectId
from tqdm import tqdm

def Tensor_power(freq,nt):
    return 8.315046112484099e-13 * np.power((10**freq)/7.731438287422856e-17, nt)

for document in tqdm(collection.find({'f': {'$exists': True}})):
    collection.update_one({'_id': ObjectId(document['_id'])}, {'$set': {
        'asymp':  list(np.array(document['log10OmegaGW']) - np.log10(Tensor_power(np.array(document['f']),document['n_t'])) + 2 * np.log10(0.6732117) - 2 * np.array(document['f']))
    }}, upsert=True)

In [ ]:
reload(stiff_SGWB)
import time
from stiff_SGWB import LCDM_SG as sg
collection = db["data_test"]
# 求解示例
times = []
for document in tqdm(collection.aggregate([{"$match":{'log10OmegaGW':{"$exists":False}}}])):
    try:
        r = document['r']
        n_t = document['n_t']
        T_re = document['T_re']
        DN_re = document['DN_re']
        kappa10 = document['kappa10']
        model4_new = sg(r=r, n_t=n_t, cr=0, T_re=T_re, DN_re=DN_re, kappa10=kappa10)
        start = time.time()
        model4_new.SGWB_iter()
        end = time.time()
        times.append(end - start)
        N_hc_arr_new = np.array(model4_new.N_hc, dtype=object)
        Th_arr_new = np.array(model4_new.Th, dtype=object)
        targets_new = [line[-1] for line in Th_arr_new]
        windows_new = [[line[0], line[-1]] for line in N_hc_arr_new]
        collection.update_one({'_id': ObjectId(document['_id'])}, {'$set': {
            "targets": targets_new,
            "windows": windows_new,
            "f":list(model4_new.f),
            "log10OmegaGW":list(model4_new.log10OmegaGW),
        }}, upsert=True)
    except AttributeError:
        continue
    # collection.insert_one(data_new)

In [4]:
times

[9.261185646057129,
 16.500418424606323,
 16.399245500564575,
 12.206757068634033,
 9.743135690689087,
 9.458157539367676,
 9.62839937210083,
 9.391486406326294,
 9.606704473495483,
 9.68180775642395,
 21.15862011909485,
 12.256763219833374,
 11.303150177001953,
 9.390976667404175,
 10.528377771377563]

In [3]:
collection = db["data3"]
for document in tqdm(collection.aggregate([{"$match":{'starts':{"$exists":False}}}])):
    collection.update_one({'_id': ObjectId(document['_id'])}, {'$set': {
        'starts':document['f'][0],
        'ends':document['f'][-1]
    }}, upsert=True)

15000it [00:08, 1863.80it/s]


In [1]:
import json
import math

with open('total.json', 'r') as f:
    data = json.load(f)

n_servers = 6
chunk_size = math.ceil(len(data) / n_servers)

for i in range(n_servers):
    start = i * chunk_size
    end = (i + 1) * chunk_size
    chunk = data[start:end]
    
    with open(f'./databatch/data_part_{i}.json', 'w') as f:
        json.dump(chunk, f, indent=2)

In [ ]:
'''
小范围扰动
'''
collection = db["macro_2"]  
T_re = 0.19453859
DN_re = 39.366618
kappa10 = 110.42477
perturbation_factor = 0.05
num_samples = 10000
for _ in tqdm(range(num_samples)):
    T_re_new = T_re * (1 + random.uniform(-0.1, 0.1))
    DN_re_new = DN_re * (1 + random.uniform(-0.5, 0))
    kappa10_new = kappa10 * (1 + random.uniform(-0.5, 0.5))
    model4_new = sg(r=3.9585109e-05, n_t=1, cr=0, T_re=T_re_new, DN_re=DN_re_new, kappa10=kappa10_new)
    model4_new.SGWB_iter()
    N_hc_arr_new = np.array(model4_new.N_hc, dtype=object)
    Th_arr_new = np.array(model4_new.Th, dtype=object)
    targets_new = [line[-1] for line in Th_arr_new]
    windows_new = [[line[0], line[-1]] for line in N_hc_arr_new]
    data_new = {
        "targets": targets_new,
        "windows": windows_new,
        "f":list(model4_new.f),
        "log10OmegaGW":list(model4_new.log10OmegaGW),
        "param": {    
            "T_re": T_re_new,
            "DN_re": DN_re_new,
            "kappa10": kappa10_new
        }
    }
    clear_output()
    collection.insert_one(data_new)

In [ ]:
'''
cache
'''
reload(stiff_SGWB)
from stiff_SGWB import LCDM_SG as sg
collection = db["param_lhs3"]
# 求解示例
for document in tqdm(collection.aggregate([{"$match":{'log10OmegaGW':{"$exists":False}}}])):
    model4_new = sg(r=3.9585109e-05, n_t=1, cr=0, T_re=document['T_re'], DN_re=document['DN_re'], kappa10=document['kappa10'])
    model4_new.SGWB_iter()
    N_hc_arr_new = np.array(model4_new.N_hc, dtype=object)
    Th_arr_new = np.array(model4_new.Th, dtype=object)
    targets_new = [line[-1] for line in Th_arr_new]
    windows_new = [[line[0], line[-1]] for line in N_hc_arr_new]
    clear_output()
    collection.update_one({'_id': ObjectId(document['_id'])}, {'$set': {
        "targets": targets_new,
        "windows": windows_new,
        "f":list(model4_new.f),
        "log10OmegaGW":list(model4_new.log10OmegaGW),
    }}, upsert=True)
    # collection.insert_one(data_new)